In [ ]:
import numpy as np

p = 3
K = GF(p)
n = 3
m = 2
# R = PolynomialRing(K, x, n) # defines n variables over K
# R.inject_variables(verbose=False) # makes all the variables ready for use

In [ ]:
# class QuadraticForm(SageObject):
# adjusted Gram matrix method that works for any ring of char != 2
def my_Gram_matrix(self):
    R = self.base_ring()
    assert R.characteristic() != 2, "Characteristic of the base ring cannot be 2."
    A = (R(1) / R(2)) * self.matrix()
    n = self.dim()

    Int_flag = True
    for i in range(n):
        for j in range(i, n):
            Int_flag &= A[i, j] in R

    # Return the Gram matrix, or an error
    if Int_flag:
        return MatrixSpace(R, n, n)(A)
    raise TypeError("this form does not have an integral Gram matrix")

QuadraticForm.my_Gram_matrix = my_Gram_matrix

In [ ]:
# given a polynomial f, output the matrix A, vector B and scalar C s.t.
# f(x) = x^T Ax + Bx + C
def quadratic_to_matrix(f):
    h = f.homogeneous_components()
    A = QuadraticForm(h.get(2)).my_Gram_matrix()
    B = vector([])
    C = 0

    # try/except but for the poor ppl 
    if h.get(1) != None:
        # we make a vector of the coefficients of each basis element
        B = vector([h.get(1).coefficient(R.gens()[i]) for i in range(n)])

    if h.get(0) != None:
        C = h.get(0)

    return A, B, C

In [ ]:
# a function that simulates precomposition of a quadratic form
# f(x) = x^T Ax + Bx + C with a n x n matrix S 
# so it returns f \circ S 
def transform_input(S, A, B, C):
    # this throws an error because S^T A S is a matrix but BS is a vector and you cannot
    # put them in together in one vector
    #
    # it works if you return them separately, not in a vector
    return S.transpose()*A*S, B*S, C

In [ ]:
def random_symmetric_matrix(K, n):
    # get a random matrix
    A = random_matrix(K, n, n)
    # and make it symmetric
    return A + A.transpose()

# returns A, B and C like above
def random_quadratic_poly(K, n):
    return random_symmetric_matrix(K, n), random_vector(K, n), K.random_element()

In [ ]:
class VectorOfMatrices:
    def __init__(self, matrices):
        self.matrices = matrices
     
    def __mul__(self, other):
        # https://doc.sagemath.org/html/en/reference/matrices/sage/matrix/matrix_dense.html
        if not isinstance(other, sage.matrix.matrix_dense.Matrix_dense):
            raise TypeError("We only multiply by a matrix my dear")

        if other.ncols() != len(self.matrices):
            raise ValueError("Matrix dimensions do not match")

        # multiply them by hand
        result = [sum(other[i, j] * self.matrices[j] for j in range(len(self.matrices))) for i in range(other.nrows())]
        # gives back a list
        return result

    def __repr__(self):
        return f"VectorOfMatrices(\n{self.matrices})"
    
    __rmul__ = __mul__

In [ ]:
# an actual example
# instead of generating an actual polynomial first 
# and then getting the corresponding matrices,
# generate the matrices randomly

# here are the actual transformations
T = matrix(GL(m, K).random_element())
S = matrix(GL(n, K).random_element())

# We define the ring with enough variables for both our matrices
R2  = PolynomialRing(K, n^2 + m^2, 'y')
R2.inject_variables(verbose=False)

# first n^2 elements of R2
S_ = matrix(R2, n, n, R2.gens()[:n^2])

# we get the last m^2 variables of R2
T_ = matrix(R2, m, m, R2.gens()[-m^2:])
# print(S_, T_)

# generate whole ass m-sized vector of (homogenous) polynomials
hom = []
for i in range(m):
    A = random_symmetric_matrix(K, n)
    hom.append(A)

In [ ]:
# this is for the homogenous part S^T AS
G = [S.transpose()*A*S for A in hom]
G_ = [S_.transpose()*A*S_ for A in hom]

v = VectorOfMatrices(G)
v_ = VectorOfMatrices(G_)

P = T*v
P_ = T_*v_

field_eqs = [y^p - y for y in R2.gens()]
system = []
for i in range(m):
    system.append(P_[i] - P[i])


flat_system = [A.list() for A in system][0]
# the list(set(.)) removes the duplicates
total_system = list(set(flat_system + field_eqs))

print(total_system)

In [ ]:
# this is for the vector part B*S


In [ ]:
I = R2.ideal(total_system)
J = R2.ideal(flat_system)
# print(I.dimension())
# J = I.groebner_basis()
Z = I.variety()
# timeit.eval("Z = I.variety()")
print(Z)

In [ ]:
L = [f.subs(v) for f in I.gens() for v in Z]
all(f.subs(v) == 0 for f in I.gens() for v in Z)

In [ ]:
from sage.rings.polynomial.msolve import variety
# help(variety)
Z = variety(I, K, proof=False)
# sorted(variety(I, K, proof=False), key=str)
print(Z)

In [ ]:
# given a system of polynomials, check if it correctly corresponds
# to the actual matrices it was created from
#
# system is a list of multivariate polynomials
# S and T are matrices
def verify_system(system, S, T):
    point = S.list()+T.list()
    return all(f(point) == 0 for f in system)